# Redis tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [Redis commands](https://redis.io/commands)
* [Redis python client](https://redis-py.readthedocs.io/en/latest/)

Though the tutorial asks you to use Redis commands, they all have an equivalent in the Python client library.

### Import libraries

In [31]:
import redis

In [32]:
r = redis.Redis(host='localhost', port=6379)

In [33]:
r.set('foo', 'bar')
value = r.get('foo')
print(value)

b'bar'


You can launch a terminal aside, connect to your server with a Redis client and check that the value is still present :

```bash
vagrant@nosql:~$ redis-cli
127.0.0.1:6379> get foo
"bar"
```

## I. Quick start

### Strings and numbers

**Q** : Store the string `hello world` in key `greetings` with a `SET` command. Then use a `GET` command to retrieve it.

In [34]:
r.set('greetings', 'hello world')
value = r.get('greetings')
print(value)


b'hello world'


**Q** : Create a `connections` key with value 0. Find the `INCR` command to increment its value, then display it. 

Try to increment the value for `greetings`.

In [35]:
r.set('connections', 0)
r.incr('connections')
value = r.get('connections')
print(value)

b'1'


In [36]:
#Try to increment the value for `greetings`.

#r.incr('greetings')
#value = r.get('greetings')
#print(value)
## Note that the value for `greetings` is not an integer, so it cannot be incremented.

### Lists

**Q** : Create a `colors` list with strings `red`, `black` and `blue`, using the `RPUSH` command.

In [37]:
r.rpush('colors', 'red')
r.rpush('colors', 'black')
r.rpush('colors', 'blue')
value = r.lrange('colors', 0, -1)
print(value)

[b'red', b'black', b'blue', b'red', b'black', b'blue']


**Q** : Display the second element of the list with `LINDEX`

In [38]:
value = r.lindex('colors', 1)
print(value)

b'black'


**Q** : Display the full list with `LRANGE`

In [39]:
value = r.lrange('colors', 0, -1)
print(value)

[b'red', b'black', b'blue', b'red', b'black', b'blue']


### Using sets

**Q** : Create a `nosql` set containing values `redis`, `mongodb` and `orientdb`, using the `SADD` command.

In [40]:
r.sadd('nosql', 'redis')
r.sadd('nosql', 'mongodb')
r.sadd('nosql', 'orientdb')

value = r.smembers('nosql')
print(value)


{b'orientdb', b'mongodb', b'hbase', b'redis'}


**Q** : Test if `mysql` is inside the set, using `SISMEMBER`

In [41]:
value = r.sismember('nosql', 'mysql')
print(value)

0


**Q** : Add `hbase` to the set with `SADD`, then display all elements of the set with `SMEMBERS`. Try to add `redis` again and see what happens.

In [42]:
r.sadd('nosql', 'hbase')
value = r.smembers('nosql')
print(value)

{b'orientdb', b'mongodb', b'hbase', b'redis'}


In [43]:
# Try to add `redis` again and see what happens.
r.sadd('nosql', 'redis')
value = r.smembers('nosql')
print(value)

{b'orientdb', b'mongodb', b'hbase', b'redis'}


### Using sorted sets

Sorted Sets are similar to Redis Sets with the unique feature of values stored in a set. The difference is, every member of a Sorted Set is associated with a score, that is used in order to take the sorted set ordered, from the smallest to the greatest score.

**Q** : Using `ZADD`, create a sorted set `top14` with the following entries :

```
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
```

In [44]:
"""
score city
10 Agen 
33 Bordeaux 
32 Brive 
29 Castres 
38 Clermont 
24 Grenoble 
26 La Rochelle 
32 Montpellier 
14 Oyonnax 
20 Pau 
40 Racing 
22 Stade Français 
36 Toulon 
36 Toulouse
"""

r.zadd('top14', {'Agen': 10, 'Bordeaux': 33, 'Brive': 32, 'Castres': 29, 'Clermont': 38, 'Grenoble': 24, 'La Rochelle': 26, 'Montpellier': 32, 'Oyonnax': 14, 'Pau': 20, 'Racing': 40, 'Stade Français': 22, 'Toulon': 36, 'Toulouse': 36})

0

**Q** : Fetch the score for `Toulon` with `ZSCORE`, and its ranking with `ZRANK`.

In [45]:
value = r.zscore('top14', 'Toulon')
print(value)

36.0


**Q** : `ZRANK` starts at 0 and scores are sorted from lowest to highest, so we should use the `ZREVRANK` for a true ranking of our cities.

In [46]:
value = r.zrevrank('top14', 'Toulon')
print(value)

3


**Q** : Find the commands to display :
* the 3 best teams
* teams with more than 35 points

In [47]:
#the 3 best teams
value = r.zrevrange('top14', 0, 2, withscores=True)
print(value)

[(b'Racing', 40.0), (b'Clermont', 38.0), (b'Toulouse', 36.0)]


In [48]:
#teams with more than 35 points
value = r.zrangebyscore('top14', 35, 50, withscores=True)
print(value)

[(b'Toulon', 36.0), (b'Toulouse', 36.0), (b'Clermont', 38.0), (b'Racing', 40.0)]


### Using dictionaries

**Q** : Create a dictionary `user:1` with `HMSET` with properties `id (1), name (Jean), age (22)`. Display it with `HGETALL`

In [49]:
value = r.hmset('user:1', {'id': 1, 'name': 'Jean', 'age': 22})
print(value)

True


C:\Users\karam\AppData\Local\Temp\ipykernel_10308\2890713463.py:1: DeprecationWarning: Redis.hmset() is deprecated. Use Redis.hset() instead.
  value = r.hmset('user:1', {'id': 1, 'name': 'Jean', 'age': 22})


**Q** : Add a `city (Lyon)` property and rename the user from `Jean` to `Paul`.

In [50]:
#for Jean
r.hset('user:1', 'city', 'Lyon')
value1 = r.hgetall('user:1')
print(value1)

{b'id': b'1', b'name': b'Jean', b'age': b'22', b'city': b'Lyon'}


In [51]:
#for Paul
r.hset('user:1', 'name', 'Paul')
value2 = r.hgetall('user:1')
print(value2)

{b'id': b'1', b'name': b'Paul', b'age': b'22', b'city': b'Lyon'}


## Modelling a query cache with Redis

You are modeling data from a REST query cache system with Redis.
A request is identified by the http method and its url (without the protocol).
The content of the request is stored as it is to be returned on demand.

**Q** : Insert a PUT request in the cache on http://my-api.fr/user/10 whose answer is {"id": 10, "name": "jean"}

In [52]:
r.set('http://my-api.fr/user/10', '{"id": 10, "name": "jean"}')

value = r.get('http://my-api.fr/user/10')
print(value)

b'{"id": 10, "name": "jean"}'


**Q** : Insert a GET request in the cache on http://my-api.fr/user?size=2 whose answer is[{"id": 10, "name": jean}, {"id": 11, "name": "Claire"}]

In [53]:
r.set('http://my-api.fr/user?size=2', '[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]')
value = r.get('http://my-api.fr/user?size=2')
print(value)

b'[{"id": 10, "name": "jean"}, {"id": 11, "name": "Claire"}]'


**Q** : Create a set of cache request keys.

In [54]:
r.sadd('cache_requests', 'http://my-api.fr/user/10')
r.sadd('cache_requests', 'http://my-api.fr/user?size=2')
value = r.smembers('cache_requests')
print(value)

{b'http://my-api.fr/user?size=2', b'http://my-api.fr/user/10'}


**Q** : Check if the GET request on http: //http://my-api.fr/user

In [55]:
value = r.sismember('cache_requests', 'http://my-api.fr/user')
print(value)

0


**Q** : Delete the PUT request on http://my-api.fr/user/10 from the cache.

In [56]:
r.delete('http://my-api.fr/user/10')
value = r.smembers('cache_requests')
print(value)

{b'http://my-api.fr/user?size=2', b'http://my-api.fr/user/10'}


## Postquisites

The folloinwg command removes all data from your Redis cluster.

In [57]:
!redis-cli flushall

'redis-cli' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
